In [1]:
!pip install transformers

"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
!pip install sacremoses

"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [3]:
import torch
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
import sacremoses

In [4]:
modelNameQa = "bert-large-uncased-whole-word-masking-finetuned-squad"
modelNameRuEn = "facebook/wmt19-ru-en"
modelNameEnRu = "facebook/wmt19-en-ru"
modelBert = "bert-base-uncased"

In [5]:
tokenizerRuEn = FSMTTokenizer.from_pretrained(modelNameRuEn)
modelRuEn = FSMTForConditionalGeneration.from_pretrained(modelNameRuEn)

tokenizerEnRu = FSMTTokenizer.from_pretrained(modelNameEnRu)
modelEnRu = FSMTForConditionalGeneration.from_pretrained(modelNameEnRu)

tokenizerQa = BertTokenizer.from_pretrained(modelNameQa)
modelQa = BertForQuestionAnswering.from_pretrained(modelNameQa)

bert_model = BertModel.from_pretrained(modelBert, output_hidden_states = True)
bert_tokenizer = BertTokenizer.from_pretrained(modelBert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Функция по переводу текста с русского на английский
def translateRuEn(inputRu):
  input = inputRu
  input_ids = tokenizerRuEn.encode(input, return_tensors="pt")
  outputs = modelRuEn.generate(input_ids)
  decodedEn = tokenizerRuEn.decode(outputs[0], skip_special_tokens=True)
  return decodedEn

In [7]:
# Функция по переводу текста с английского на русский
def translateEnRu(inputEn):
  input = inputEn
  input_ids = tokenizerEnRu.encode(input, return_tensors="pt")
  outputs = modelEnRu.generate(input_ids)
  decodedRu = tokenizerEnRu.decode(outputs[0], skip_special_tokens=True)
  return decodedRu

#Проверка работоспособности

In [8]:
question = "люди были на луне?"
answer_text = "В период с 1969 по 1972 год по программе «Аполлон» было выполнено 6 полётов с посадкой на Луне. Всего на Луне высаживались 12 астронавтов США. Список космонавтов Список космонавтов — участников орбитальных космических полётов Список астронавтов США — участников орбитальных космических полётов Список космонавтов СССР и России — участников космических полётов Список женщин-космонавтов Список космонавтов, посещавших МКС Энциклопедия астронавтики"


In [9]:
# функция, которая производит сопоставление вопроса от пользователя с сохраненным в системе

def answer_question(questionRu, answer_textRu):

    question = translateRuEn(questionRu)
    answer_text = translateRuEn(answer_textRu)

    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizerQa.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizerQa.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    output = modelQa(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    start_scores = output.start_logits
    end_scores = output.end_logits
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizerQa.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    # print(translateEnRu(answer))
    return translateEnRu(answer)

In [10]:
answer_question(question, answer_text)

'Всего с 1969 по 1972 год на Луну высадились 12 американских астронавтов'

In [11]:
def getAnswer(questionFromUser, dataJSON,qEn):

    questionFromUserEn = translateRuEn(questionFromUser)
    queryArray = []

    for i in qEn:
      text_query = i
      text_A = questionFromUserEn

      emb_query = get_embedding(bert_model, bert_tokenizer, text_query)
      emb_A = get_embedding(bert_model, bert_tokenizer, text_A)

      sim_query_A = 1 - cosine(emb_query, emb_A)
      queryArray.append(sim_query_A)

    lst_num = list(enumerate(queryArray, 0))
    t_max = max(lst_num, key=lambda i : i[1])

    questionTotal = dataJSON.question[t_max[0]]
    passageTotal = dataJSON.passage[t_max[0]]


    return answer_question(questionFromUser, passageTotal)

In [12]:
def get_word_indeces(tokenizer, text, word):
    word_tokens = tokenizer.tokenize(word)
    masks_str = ' '.join(['[MASK]']*len(word_tokens))
    text_masked = text.replace(word, masks_str)
    input_ids = tokenizer.encode(text_masked)
    mask_token_indeces = np.where(np.array(input_ids) == tokenizer.mask_token_id)[0]
    return mask_token_indeces

def get_embedding(b_model, b_tokenizer, text, word=''):
    if not word == '':
        word_indeces = get_word_indeces(b_tokenizer, text, word)
    encoded_dict = b_tokenizer.encode_plus(
                        text,                      # текст предложения.
                        add_special_tokens = True, # спец-токены '[CLS]' и '[SEP]'
                        return_tensors = 'pt',     # возвращаемый результат: тензоры pytorch
                )

    input_ids = encoded_dict['input_ids']
    b_model.eval()
    bert_outputs = b_model(input_ids)

    with torch.no_grad():

        outputs = b_model(input_ids)
        hidden_states = outputs[2]

    token_vecs = hidden_states[-2][0]

    sentence_embedding = torch.mean(token_vecs, dim=0)
    sentence_embedding = sentence_embedding.detach().numpy()

    if not word == '':
        word_embedding = torch.mean(token_vecs[word_indeces], dim=0)
        word_embedding = word_embedding.detach().numpy()

        return (sentence_embedding, word_embedding)
    else:
        return sentence_embedding

In [13]:
dataJSON = pd.read_json('./dataset.json')
qEn = pd.read_csv('./qEnShort.csv',header = None)[1].values

In [14]:
questionFromUser = 'может ли автомобиль ездить на газу?'
answer = getAnswer(questionFromUser, dataJSON,qEn)
answer

'они могут использовать как сжатый природный газ, так и бензин в качестве моторного топлива'